## Patch effectiveness

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from math import ceil
import logging
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
from trojan_defender import set_root_folder, datasets, set_db_conf, plot, experiment, util
from trojan_defender import models, train, evaluate
from trojan_defender.poison import patch
from trojan_defender.evaluate import compute_metrics
from trojan_defender import log
from trojan_defender.detect import saliency_ as saliency
from sklearn.metrics import classification_report, accuracy_score
from sklearn.covariance import EllipticEnvelope
from scipy import stats

/Users/Edu/miniconda3/envs/trojan/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# config logging
logging.basicConfig(level=logging.INFO)

# matplotlib size
plt.rcParams['figure.figsize'] = (10, 10)

# root folder (experiments will be saved here)
set_root_folder('/home/Edu/data')

# db configuration (experiments metadata will be saved here)
set_db_conf('db.yaml')

dataset_name = 'mnist'
objective_class = 5
METRICS = [accuracy_score]

ValueError: /home/Edu/data does not exist

In [4]:
conf = get_db_conf()
client = MongoClient(conf['uri'])
con = client[conf['db']][conf['collection']]

In [ ]:
# baseline
baseline = con.find({'dataset.name': 'MNIST',
                     'group_name': 'exp-static',
                     'metrics.patch_success_rate': {"$exists": False}})

# attacks
attacked = con.find({'dataset.name': 'MNIST',
                     'group_name': 'exp-static',
                     'metrics.patch_success_rate': {"$exists": True}})